In [32]:
import pandas as pd
import datetime
import numpy as np
import statsmodels.formula.api as sm

In [3]:
df = pd.read_csv('testData.csv', header=None)
df.head()

,0,1,2,3,4,5
0,42,OPEN,8,22,2018-03-13 15:22:45,NaN
1,30,OPEN,5,15,2018-03-13 15:22:20,NaN
2,54,OPEN,8,25,2018-03-13 15:23:01,NaN
3,56,OPEN,30,10,2018-03-13 15:22:42,NaN
4,6,OPEN,5,15,2018-03-13 15:23:30,NaN


In [4]:
df.shape

(228000, 6)

In [5]:
df1 = pd.read_csv('dublin_weather.csv')
df1.head()

,city_id,clouds.all,dt,dt_iso,main.humidity,main.pressure,main.temp,main.temp_max,main.temp_min,rain.1h,rain.24h,rain.3h,wind.deg,wind.speed,weather.description,weather.icon,weather.id,weather.main
0,2964574,75,1514764800,2018-01-01 00:00:00 +0000 UTC,80,990,277.15,277.15,277.15,NaN,NaN,NaN,240,11,broken clouds,04n,803,Clouds
1,2964574,75,1514768400,2018-01-01 01:00:00 +0000 UTC,80,990,277.58,278.15,277.15,NaN,NaN,NaN,240,13,light rain,10n,500,Rain
2,2964574,40,1514772000,2018-01-01 02:00:00 +0000 UTC,86,990,277.15,277.15,277.15,NaN,NaN,NaN,240,10,scattered clouds,03n,802,Clouds
3,2964574,40,1514775600,2018-01-01 03:00:00 +0000 UTC,80,990,277.58,278.15,277.15,NaN,NaN,NaN,250,12,light intensity shower rain,09n,520,Rain
4,2964574,40,1514779200,2018-01-01 04:00:00 +0000 UTC,80,990,277.58,278.15,277.15,NaN,NaN,NaN,240,9,scattered clouds,03n,802,Clouds


In [6]:
df1.shape

(2203, 18)

In [7]:
df_weather = df1[['dt_iso', 'main.temp', 'weather.main']]
df_weather.head(5)

,dt_iso,main.temp,weather.main
0,2018-01-01 00:00:00 +0000 UTC,277.15,Clouds
1,2018-01-01 01:00:00 +0000 UTC,277.58,Rain
2,2018-01-01 02:00:00 +0000 UTC,277.15,Clouds
3,2018-01-01 03:00:00 +0000 UTC,277.58,Rain
4,2018-01-01 04:00:00 +0000 UTC,277.58,Clouds


In [8]:
HourList = []
for row in df_weather.iterrows():
    split = row[1]['dt_iso'].split(' ')
    hour = (split[1].split(':'))
    HourList.append(hour[0])
#HourList

In [9]:
df_Hour = pd.DataFrame({'Hour':HourList})
df_Hour['Hour'] = df_Hour['Hour'].astype('int64')

df_new = pd.concat([df_weather, df_Hour], axis=1)
df_new.head(5)              

,dt_iso,main.temp,weather.main,Hour
0,2018-01-01 00:00:00 +0000 UTC,277.15,Clouds,0
1,2018-01-01 01:00:00 +0000 UTC,277.58,Rain,1
2,2018-01-01 02:00:00 +0000 UTC,277.15,Clouds,2
3,2018-01-01 03:00:00 +0000 UTC,277.58,Rain,3
4,2018-01-01 04:00:00 +0000 UTC,277.58,Clouds,4


In [10]:
DayList = []
for row in df_weather.iterrows():
    split = row[1]['dt_iso'].split(' ')
    DayList.append(split[0])
#DayList

In [11]:
df_Day = pd.DataFrame({'Day':DayList})
df_Day['Day'] = df_Day['Day'].astype('str')

df_Days = pd.concat([df_new, df_Day], axis=1)
df_Days.head(5)

,dt_iso,main.temp,weather.main,Hour,Day
0,2018-01-01 00:00:00 +0000 UTC,277.15,Clouds,0,2018-01-01
1,2018-01-01 01:00:00 +0000 UTC,277.58,Rain,1,2018-01-01
2,2018-01-01 02:00:00 +0000 UTC,277.15,Clouds,2,2018-01-01
3,2018-01-01 03:00:00 +0000 UTC,277.58,Rain,3,2018-01-01
4,2018-01-01 04:00:00 +0000 UTC,277.58,Clouds,4,2018-01-01


In [12]:
df_Days.groupby('weather.main')['weather.main'].apply(lambda x: x.count())

weather.main
Clear        59
Clouds     1350
Drizzle     181
Fog          44
Mist         55
Rain        382
Snow        132
Name: weather.main, dtype: int64

In [13]:
df_Days = df_Days.drop(['dt_iso'], axis=1)
df_Days.head(5)

,main.temp,weather.main,Hour,Day
0,277.15,Clouds,0,2018-01-01
1,277.58,Rain,1,2018-01-01
2,277.15,Clouds,2,2018-01-01
3,277.58,Rain,3,2018-01-01
4,277.58,Clouds,4,2018-01-01


In [14]:
RainList = []
for row in df_Days['weather.main'].iteritems():
    if row[1] == 'Rain' or row[1] == 'Drizzle':
        RainList.append(1)
    else:
        RainList.append(0)
#RainList

In [15]:
df_Rain = pd.DataFrame({'Rain':RainList })
df_Rain['Rain'] = df_Rain['Rain'].astype('bool')
CleanedDF = pd.concat([df_Days, df_Rain], axis=1)
final_df = CleanedDF.tail(485)
final_df.head(5)

,main.temp,weather.main,Hour,Day,Rain
1718,282.15,Clouds,15,2018-03-13,False
1719,282.15,Clouds,16,2018-03-13,False
1720,282.15,Clouds,17,2018-03-13,False
1721,282.15,Clouds,18,2018-03-13,False
1722,280.72,Clouds,19,2018-03-13,False


In [16]:
# now for the test
df.head(5)

,0,1,2,3,4,5
0,42,OPEN,8,22,2018-03-13 15:22:45,NaN
1,30,OPEN,5,15,2018-03-13 15:22:20,NaN
2,54,OPEN,8,25,2018-03-13 15:23:01,NaN
3,56,OPEN,30,10,2018-03-13 15:22:42,NaN
4,6,OPEN,5,15,2018-03-13 15:23:30,NaN


In [17]:
HourList_test = []
for row in df.iterrows():
    split = row[1][4].split(' ')
    hour = (split[1].split(':'))
    HourList_test.append(hour[0])
#HourList_test

['15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '16',
 '16',
 '16',
 '16',
 '16',
 '16',
 '16',
 '16',
 '16',
 '16',
 '16',
 '16',
 '16',
 '16',
 '16',
 '16',
 '16',
 '16',
 '16',
 '16',
 '16',
 '16',
 '16',
 '16',
 '16',
 '16',
 '16',
 '16',
 '16',
 '16',
 '16',
 '16',
 '16',
 '16',
 '16',
 '16',
 '16',
 '16',
 '16',
 '16',
 '16',
 '16',
 '16',

In [18]:
df_HourTest = pd.DataFrame({'Hour':HourList_test})
df_HourTest['Hour'] = df_HourTest['Hour'].astype('int64')

df_newTest = pd.concat([df, df_HourTest], axis=1)
df_newTest.tail(5)
df_newTest.dtypes

0         int64
1        object
2         int64
3         int64
4        object
5       float64
Hour      int64
dtype: object

In [19]:
DayList_Test = []
for row in df_newTest.iterrows():
    split = row[1][4].split(' ')
    DayList_Test.append(split[0])
#DayList_Test

In [20]:
df_Test = pd.DataFrame({'Day': DayList_Test})
df_Test['Day'] = df_Test['Day'].astype('str')

df_Testing = pd.concat([df_Test, df_newTest], axis=1)
df_Testing.tail(5)

,Day,0,1,2,3,4,5,Hour
227995,2018-03-27,28,OPEN,16,14,2018-03-27 11:24:37,NaN,11
227996,2018-03-27,39,OPEN,12,7,2018-03-27 11:31:50,NaN,11
227997,2018-03-27,83,OPEN,0,40,2018-03-27 11:31:55,NaN,11
227998,2018-03-27,92,OPEN,0,40,2018-03-27 11:29:05,NaN,11
227999,2018-03-27,21,OPEN,29,0,2018-03-27 11:25:22,NaN,11


In [21]:
df_Testing = df_Testing.drop([4], axis=1)
df_Testing = df_Testing.drop([1], axis=1)
df_Testing = df_Testing.drop([5], axis=1)

df_Testing.tail(5)

,Day,0,2,3,Hour
227995,2018-03-27,28,16,14,11
227996,2018-03-27,39,12,7,11
227997,2018-03-27,83,0,40,11
227998,2018-03-27,92,0,40,11
227999,2018-03-27,21,29,0,11


In [22]:
df_Testing.columns = ['Day','station','available_bikes', 'available_stands', 'Hour']
df_Testing.tail(5)

,Day,station,available_bikes,available_stands,Hour
227995,2018-03-27,28,16,14,11
227996,2018-03-27,39,12,7,11
227997,2018-03-27,83,0,40,11
227998,2018-03-27,92,0,40,11
227999,2018-03-27,21,29,0,11


In [23]:
df_finished = df_Testing.merge(final_df, how='inner', on = ['Day', 'Hour'])

In [24]:
df_finished = df_finished.iloc[::20]
df_finished.shape

(11400, 8)

In [25]:
for i in df_finished:
    print(i)

Day
station
available_bikes
available_stands
Hour
main.temp
weather.main
Rain


In [28]:
percentage = []
for row in df_finished.iterrows():
    figure = df_finished['available_bikes'] / (df_finished['available_bikes'] + df_finished['available_stands'])
    percentage.append(figure)
#percentage

In [29]:
df_percent = pd.DataFrame({'Percent': percentage[1]})
df_percent['Percent'] = df_percent['Percent'].astype('float')

df_fin = pd.concat([df_finished, df_percent], axis=1)
df_fin.tail(5)

,Day,station,available_bikes,available_stands,Hour,main.temp,weather.main,Rain,Percent
227900,2018-03-27,42,4,26,11,281.64,Drizzle,True,0.133333
227920,2018-03-27,84,0,29,11,281.64,Drizzle,True,0.000000
227940,2018-03-27,19,28,0,11,281.64,Drizzle,True,1.000000
227960,2018-03-27,51,20,20,11,281.64,Drizzle,True,0.500000
227980,2018-03-27,100,0,25,11,281.64,Drizzle,True,0.000000


In [35]:
# Build a learning machine using the subset of features chosen in part 1
lm = sm.ols(formula="Percent ~ Rain", data=df_fin).fit()

# Print the model parameters (e.g coefficients)
print(lm.params)

Intercept       0.460286
Rain[T.True]    0.022510
dtype: float64
